<a href="https://colab.research.google.com/github/forwardyoung/TIL/blob/master/autogloun_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon

In [2]:
# from google.colab import files
# import os
# os.listdir('.')
# # uploaded = files.upload()

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

# TabularDataset을 사용하여 데이터를 불러오기
train_data = TabularDataset('train.csv')
test_data = TabularDataset('test.csv')

In [4]:
# ID 컬럼은 단순히 각 데이터를 구분하기 위한 숫자나 문자열로, 예측하려는 목표 변수와는 아무런 상관이 없음.
# 음악 장르를 예측하는 데 있어서 노래의 ID는 아무런 영향을 미치지 않기 때문에 ID 컬럼 제거

# axis = 0 => row(행)
# axis = 1 => column(열)

train_data.drop("ID", axis=1, inplace=True)
test_data.drop("ID", axis=1, inplace=True)

In [5]:
# 모델 학습

# 모델이 예측해야 하는 타겟 변수(레이블)
label = 'genre'
# 모델의 성능을 평가하는 데 사용할 메트릭을 지정
eval_metric = 'f1_macro'

time_limit = 360

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
    ).fit(train_data, presets='best_quality', time_limit=time_limit)
# fit() : TabularPredictor 클래스에서 모델을 학습시키는 데 사용됨
# presets='best_quality' 최상의 품질 모델을 찾기 위한 사전 설정을 사용하겠다는 의미

No path specified. Models will be saved in: "AutogluonModels/ag-20240801_061356"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.50 GB / 12.67 GB (90.7%)
Disk Space Avail:   71.57 GB / 107.72 GB (66.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

In [6]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
ld_board = predictor.leaderboard(train_data, silent=True)
ld_board

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,0.954676,0.335610,f1_macro,0.358550,0.395540,0.132884,0.358550,0.395540,0.132884,1,True,2
1,WeightedEnsemble_L2,0.669798,0.636354,f1_macro,5.519031,1.149994,161.310594,0.009454,0.020246,2.275470,2,True,4
2,NeuralNetFastAI_BAG_L1,0.660612,0.636138,f1_macro,5.263303,0.733536,158.899031,5.263303,0.733536,158.899031,1,True,3
3,WeightedEnsemble_L3,0.643891,0.653573,f1_macro,9.517942,2.873657,253.660875,0.007003,0.020094,2.583372,3,True,6
4,NeuralNetFastAI_BAG_L2,0.638571,0.652737,f1_macro,9.510939,2.853563,251.077504,3.642813,1.328275,91.909496,2,True,5
5,KNeighborsUnif_BAG_L1,0.512996,0.288208,f1_macro,0.246274,0.396212,0.136093,0.246274,0.396212,0.136093,1,True,1


In [9]:
# 가장 성능이 좋은 모델
best_model = predictor.model_best

# 예측 수행
model_pred = predictor.predict(test_data, model=best_model)

In [11]:
# 결과 저장
import pandas as pd

# 제출 파일 읽기
submission = pd.read_csv('sample_submission.csv')

submission['genre'] = model_pred
submission.head()

# 제출 파일 저장
submission.to_csv('sample_submission.csv', index=False)